In [1]:
import os
import sys
import time
import datetime
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['font.family']=['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
from collections import Counter
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import summary_statistics as ss
from data_io_utils import *

In [2]:
date = datetime.date.today().strftime('%m-%d')
file_path= os.path.join('/Users/yantingting/Seafile/风控/模型/10 印尼/低费率测试/ytt/', date)
if not os.path.exists(file_path):
    os.mkdir(file_path)

In [3]:
query1 = '''
SELECT
		c.level_group,
		c.campaign,
		substring(c.create_time::varchar, 1, 10) as createtime,
		substring(l.apply_time::varchar, 1, 10) as applytime,
		count(distinct c.id) as customer_install ,
		count(distinct l.id) as application ,
		count(distinct case when r.risklevel is not null and r.risklevel <> '' then l.id end) as RC ,
		count(distinct case when r.basicrulesnode = 'R' then l.id end) as basicrulesnode,
	    count(distinct case when r.modelnode = 'R' then l.id end) as modelnode,
	    count(distinct case when r.thirdpartrulenode = 'R' then l.id end) as thirdpartrulenode,
	    count(distinct case when r.thirdpartblacklist = 'R' then l.id end) as thirdpartblacklist,
		count(distinct case when r.risklevel in ('N', 'P') then l.id end) as RC_pass ,
		count(distinct case when photo_result is not null and photo_result <> '' and r.risklevel in ('N', 'P') then l.id end) as photo,
		count(distinct case when photo_result = 'ENABLE' and photo_result <> '' and r.risklevel in ('N', 'P') then l.id end) as photo_pass,
		count(distinct case when bank_card_result is not null and bank_card_result <> '' and r.risklevel in ('N', 'P') then l.id end) as bank ,
		count(distinct case when bank_card_result in('BIND', 'BINDING') and r.risklevel in ('N', 'P') and bank_card_result <> '' then l.id end) as bank_pass ,
		count(distinct case when bank_card_result in('BIND', 'BINDING') and photo_result = 'ENABLE' and r.risklevel in ('N', 'P') then l.id end) as bankandphoto_pass,
		count(distinct case when r.risklevel in ('N', 'P') and human_trial = 'HUMAN_TRIAL' then l.id end) as audit2,
		count(distinct case when r.risklevel in ('N', 'P') and human_trial = 'HUMAN_TRIAL' and loan_status not in ('DENIED', 'RESCIND', 'APPROVING', 'CREATED') then l.id end) as audit2pass,
		count(distinct case when effective_date >= '2020-01-07' and loan_status not in ('DENIED', 'RESCIND', 'APPROVING', 'CREATED') then l.id else null end)as fangkuan,
		count(distinct case when r.existsloanwithcontacts = 'reject' then l.id end) as existsloanwithcontacts ,
		count(distinct case when r.installdangerapp = 'reject' then l.id end) as installdangerapp ,
		count(distinct case when r.callwithbadguys = 'reject' then l.id end) as callwithbadguys ,
		count(distinct case when r.blacklist = 'reject' then l.id end) as blacklist ,
		count(distinct case when r.invalidage = 'reject' then l.id end) as invalidage ,
		count(distinct case when r.deniedbyoccupation = 'reject' then l.id end) as deniedbyoccupation ,
		count(distinct case when r.installspecifyapp = 'reject' then l.id end) as installspecifyapp ,
		count(distinct case when r.deniedloanwith5fields = 'reject' then l.id end) as deniedloanwith5fields ,
		count(distinct case when r.duplicatecustomerwith5fields = 'reject' then l.id end) as duplicatecustomerwith5fields ,
		count(distinct case when r.existsloanwith5fields = 'reject' then l.id end) as existsloanwith5fields ,
		count(distinct case when r.bankantifraud = 'reject' then l.id end) as bankantifraud ,
		count(distinct case when r.worklocationrestrict = 'reject' then l.id end) as worklocationrestrict ,
		count(distinct case when r.basicrulesnode in ('N','P') then l.id end) as  basicrulesnode_pass,
		count(distinct case when r.basicrulesnode='R' then l.id end) as  basicrulesnode_reject,
		count(distinct case when r.newusermodelresultv5 = 'reject' then l.id end) as newusermodelresultv5 ,
		count(distinct case when r.denyiziinquiriesbytypetotal = 'reject' then l.id end) as denyiziinquiriesbytypetotal ,
		count(distinct case when r.denyiziphoneage = 'reject' then l.id end) as denyiziphoneage ,
		count(distinct case when r.thirdpartrulenode in ('N','P') then l.id end) as  thirdpartrulenode_pass,
		count(distinct case when r.thirdpartrulenode='R' then l.id end) as  thirdpartrulenode_reject,
		count(distinct case when r.ruhiablacklist = 'reject' then l.id end) as ruhiablacklist ,
		count(distinct case when r.xhprcblacklist = 'reject' then l.id end) as xhprcblacklist ,
		count(distinct case when r.xhprcgreylist = 'reject' then l.id end) as xhprcgreylist,
		count(distinct case when r.thirdpartblacklist in ('N','P') then l.id end) as  thirdpartblacklist_pass,
		count(distinct case when r.thirdpartblacklist='R' then l.id end) as  thirdpartblacklist_reject
	from (select * from dw_gocash_go_cash_loan_gocash_core_customer where device = 'ANDROID' and level_group != ''	and product_code='UKU') c
	left join dw_gocash_go_cash_loan_gocash_core_loan l on c.id = l.customer_id
	left join rt_risk_mongo_gocash_installmentriskcontrolresult r on l.id::varchar = r.loanid
	left join risk_mongo_gocash_installmentriskcontrolparams p on p.riskno = r.riskno
	group by 1,2,3,4
'''

In [5]:
df1  = data_con.database_gn(query1) 
df1.shape
save_data_to_pickle(df1, file_path, '全流程转化.pkl')
df1.head()

Successful connection


(275651, 44)

,level_group,campaign,createtime,applytime,customer_install,application,rc,basicrulesnode,modelnode,thirdpartrulenode,thirdpartblacklist,rc_pass,photo,photo_pass,bank,bank_pass,bankandphoto_pass,audit2,audit2pass,fangkuan,existsloanwithcontacts,installdangerapp,callwithbadguys,blacklist,invalidage,deniedbyoccupation,installspecifyapp,deniedloanwith5fields,duplicatecustomerwith5fields,existsloanwith5fields,bankantifraud,worklocationrestrict,basicrulesnode_pass,basicrulesnode_reject,newusermodelresultv5,denyiziinquiriesbytypetotal,denyiziphoneage,thirdpartrulenode_pass,thirdpartrulenode_reject,ruhiablacklist,xhprcblacklist,xhprcgreylist,thirdpartblacklist_pass,thirdpartblacklist_reject
0,ControlGroup,,2017-12-01 22,2020-01-22 16,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0
1,ControlGroup,,2017-12-03 00,None,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ControlGroup,,2017-12-04 11,None,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ControlGroup,,2017-12-04 17,None,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ControlGroup,,2017-12-05 09,None,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
df1 = load_data_from_pickle(file_path,'全流程转化.pkl')
df1 = df1[(df1['level_group']!='OneTest2') & (df1['level_group']!='OneTest1')&(df1['level_group']!='OneControlGroup')]
df1['reg_date'] = df1['createtime'].apply(lambda x: x[:10])
df1.to_excel(os.path.join(file_path, 'temp.xlsx'))
print(10)

In [5]:
frame1 = pd.pivot_table(df1, index = 'reg_date',columns = 'level_group', values=('customer_install'), aggfunc='sum',fill_value=0)
frame1['all'] = frame1.apply(lambda x: x.sum(), axis = 1)
list1 = ['ControlGroup',	'Test1',	'Test2', 'Test3', 'Test4']
for cols in list1:
    frame1[cols + '1'] = frame1[cols]/frame1['all']
frame1

level_group,ControlGroup,Test1,Test2,Test3,Test4,all,ControlGroup1,Test11,Test21,Test31,Test41
reg_date,,,,,,,,,,,
2017-11-26,0,0,0,1,0,1,0.000000,0.000000,0.000000,1.000000,0.000000
2017-11-30,0,3,0,0,2,5,0.000000,0.600000,0.000000,0.000000,0.400000
2017-12-01,1,2,1,0,1,5,0.200000,0.400000,0.200000,0.000000,0.200000
2017-12-03,1,1,0,2,0,4,0.250000,0.250000,0.000000,0.500000,0.000000
2017-12-04,2,2,2,0,2,8,0.250000,0.250000,0.250000,0.000000,0.250000
2017-12-05,2,2,1,0,1,6,0.333333,0.333333,0.166667,0.000000,0.166667
2017-12-06,0,1,3,1,1,6,0.000000,0.166667,0.500000,0.166667,0.166667
2017-12-07,0,1,3,0,0,4,0.000000,0.250000,0.750000,0.000000,0.000000
2017-12-08,1,1,1,2,1,6,0.166667,0.166667,0.166667,0.333333,0.166667
